In [1]:
%run "Setup_Env.ipynb"

## Routing Chains with LCEL

The idea here is to have multiple individual LLM Chains which can perform their own tasks like summarize, sentiment etc.

We also have a router chain which can classify the user prompt intent and then route the user prompt to the relevant LLM Chain e.g if the user wants to summarize an article, his prompt request would be routed to the summarize chain automatically to get the result

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [5]:
sample_review = """
    Required a stylish lamp for my office space, and this particular one
    came with added storage at a reasonable price.
    The delivery was surprisingly quick, arriving within just two days.
    However, the pull string for the lamp suffered damage during transit.
    To my relief, the company promptly dispatched a replacement,
    which arrived within a few days. Assembly was a breeze.
    Then, I encountered an issue with a missing component,
    but their support team responded swiftly and provided the missing part.
    It appears to be a commendable company that genuinely values its
    customers and the quality of its products.
"""

In [4]:
classifier_prompt = ChatPromptTemplate.from_template(
        """Given the user instructions below for analyzing customer review,
           classify it as only one of the following categories:
            - summarize
            - sentiment
            - email

          Do not respond with more than one word.

          Instructions:
          {instruction}
""")

classifier_chain = (
    classifier_prompt
        |
    chatgpt
        |
    StrOutputParser()
)

In [7]:
classifier_chain.invoke({"instruction": "Provide the Summary of the following review"})

'summarize'

In [8]:
summary_prompt = ChatPromptTemplate.from_template(
    """Act as a customer review analyst, given the following customer review,
       generate a short summary (max 2 lines) of the review.

       Customer Review:
       {review}
""")

summary_chain = (
    summary_prompt
        |
    chatgpt
        |
    StrOutputParser()
)

In [10]:
print(summary_chain.invoke({"review": sample_review}))


The customer praised the stylish lamp with added storage and quick delivery, despite initial issues with a damaged pull string and a missing component, which were promptly resolved by the supportive company.


In [11]:
sentiment_prompt = ChatPromptTemplate.from_template(
    """Act as a customer review analyst, given the following customer review,
       find out the sentiment of the review.
       The sentiment can be either positive, negative or neutral.
       Return the result as a single word.

       Customer Review:
       {review}
""")

sentiment_chain = (
    sentiment_prompt
        |
    chatgpt
        |
    StrOutputParser()
)

In [12]:
print(sentiment_chain.invoke({"review": sample_review}))


Positive


In [13]:
email_prompt = ChatPromptTemplate.from_template(
    """Act as a customer review analyst,
    given the following customer review and its sentiment
    generate an email response to the customer based on the following conditions.
     - If the sentiment is positive or neutral thank them for their review
     - If the sentiment is negative, apologize to them

    Customer Review:
    {review}
    Sentiment:
    {sentiment}
""")

email_chain = (
    email_prompt
        |
    chatgpt
        |
    StrOutputParser()
)

In [15]:
print(email_chain.invoke({"review": sample_review, "sentiment": "positive"}))

Subject: Thank You for Your Review!

Dear [Customer's Name],

Thank you so much for taking the time to share your experience with us! We’re thrilled to hear that you found the lamp stylish and that it met your needs for your office space. It’s great to know that our team was able to assist you promptly with the replacement and the missing component.

Your feedback is invaluable to us, and we truly appreciate your kind words about our commitment to customer satisfaction. If you have any further questions or need assistance in the future, please don’t hesitate to reach out.

Thank you once again for choosing our products!

Best regards,

[Your Name]  
[Your Position]  
[Company Name]  
[Contact Information]  


In [16]:
def default_answer(query):
  return "Sorry instructions are not the defined intents"

In [18]:
from langchain_core.runnables import RunnableParallel, RunnableBranch

branch = RunnableBranch(
    (lambda x: "summarize" in x["topic"].lower(), summary_chain),
    (lambda x: "sentiment" in x["topic"].lower(), sentiment_chain),
    (lambda x: "email" in x["topic"].lower(), email_chain),
    default_answer,
)

In [19]:
full_chain = ({
                "topic": classifier_chain,
                "instruction": lambda input_prompt: input_prompt.get("instruction"),
                "review": lambda input_prompt: input_prompt.get("review"),
                "sentiment": lambda input_prompt: input_prompt.get("sentiment")

              }
                  |
                branch)

In [20]:
summary = full_chain.invoke({"instruction": "Generate a summary for the given review",
                   "review": sample_review})
print(summary)

The customer praised the stylish lamp with added storage and quick delivery, despite initial issues with a damaged pull string and a missing component, which were promptly resolved by the supportive company.


In [21]:
sentiment = full_chain.invoke({"instruction": "Find out the sentiment of the customer in the review",
                   "review": sample_review})
print(sentiment)

Positive


In [22]:
response = full_chain.invoke({"instruction": "Write an email for the given customer review",
                              "review": sample_review,
                              "sentiment": sentiment})
print(response)

Subject: Thank You for Your Review!

Dear [Customer's Name],

Thank you so much for taking the time to share your experience with us! We’re thrilled to hear that you found the lamp stylish and that it met your needs for your office space. It’s great to know that our delivery service was prompt and that our support team was able to assist you quickly with the replacement and missing component.

Your kind words about our commitment to customer satisfaction mean a lot to us. We strive to provide quality products and excellent service, and it’s wonderful to see that reflected in your experience.

If you have any further questions or need assistance in the future, please don’t hesitate to reach out. We’re here to help!

Best regards,

[Your Name]  
[Your Position]  
[Company Name]  
[Contact Information]  
